# 🎼 Music Generation with MusicGen

In this notebook, you'll generate short music clips using **text prompts** and Meta's MusicGen models. You’ll be able to describe a musical style or mood — like `"A calm piano melody with rain in the background"` — and the model will generate a unique audio sample for you.

You can choose between two model sizes:
- **Small (300M)** – Fast, but lower quality and less expressive.
- **Medium (1.5B)** – Better sound quality and more musical nuance, but **much slower**, especially for longer clips.

⚠️ **Tip:** If you select the **medium model** and a clip length longer than 15 seconds, generation may take **up to several minutes**. Start small if you're just experimenting!

Once your clip is ready, you can listen to it directly in the notebook 🎧

In [ ]:
import gradio as gr
from transformers import pipeline
import scipy

# Model selection options
MODEL_OPTIONS = {
    "Small (300M)": "facebook/musicgen-small",
    "Medium (1.5B)": "facebook/musicgen-medium"
}

def generate_music(prompt, model_name, length_seconds):
    model_id = MODEL_OPTIONS[model_name]
    synthesiser = pipeline("text-to-audio", model=model_id)
    
    # Token estimation: ~50 tokens per second
    max_tokens = int(length_seconds * 50)
    params = {"do_sample": True, "max_new_tokens": max_tokens}
    
    out = synthesiser(prompt, forward_params=params)
    audio = out["audio"][0] if isinstance(out["audio"], list) else out["audio"]
    sr = out["sampling_rate"]
    
    scipy.io.wavfile.write("music.wav", rate=sr, data=audio)
    return sr, audio

gr.Interface(
    fn=generate_music,
    inputs=[
        gr.Textbox(lines=2, label="Prompt", placeholder="e.g. Energetic synthwave track with retro vibes"),
        gr.Dropdown(list(MODEL_OPTIONS.keys()), value="Small (300M)", label="Model"),
        gr.Slider(minimum=5, maximum=30, step=1, value=10, label="Clip Length (seconds)")
    ],
    outputs=gr.Audio(type="numpy", label="Generated Music"),
    title="🎶 MusicGen Text‑to‑Music",
    description="Generate music from text using Facebook's MusicGen. Choose your model and clip length."
).launch(server_name="0.0.0.0", server_port=8080, share=True)